In [2]:
# reference: http://cs229.stanford.edu/notes/cs229-notes1.pdf
import numpy as np
#import matplotlib as ml
#import matplotlib.pyplot as plt
import random
import sys

data = np.loadtxt('assign1_data.txt')
#print data

x1 = data[:, 0]
print "x1:", x1

x2 = data[:, 1]
print "x2:", x2

y = data[:, 2]
#print "y:", y

z = data[:, 3]
#print "z:", z

'''
correct solution from part1 is:
w1:	-2.04424259514
w2:	3.99686016866
b:	-0.924290811868
'''

x1: [ 0.1227  0.3914  0.7725  0.8342  0.5084  0.9983  0.6643  0.0954  0.6949
  0.6075  0.3717  0.0193  0.9314  0.2206  0.0903  0.5466  0.4331  0.5293
  0.8955  0.0701  0.0771  0.1972  0.6405  0.3203  0.103   0.5605  0.4288
  0.7101  0.6575  0.9498  0.555   0.0947  0.4293  0.31    0.7297  0.1973
  0.1571  0.2272  0.2099  0.3972  0.0033  0.6441  0.5549  0.9193  0.5341
  0.1555  0.1839  0.9949  0.7918  0.4088  0.2247  0.3901  0.4309  0.0732
  0.8077  0.0854  0.8955  0.8794  0.1094  0.0726  0.4377  0.889   0.5791
  0.9721  0.5719  0.3156  0.8065  0.4777  0.6264  0.2117  0.042   0.0916
  0.6035  0.6354  0.4102  0.695   0.2153  0.0768  0.2356  0.1591  0.7171
  0.9988  0.3936  0.0201  0.6295  0.1771  0.5888  0.5302  0.3434  0.7154
  0.3506  0.5432  0.4687  0.013   0.1311  0.697   0.3996  0.2208  0.1119
  0.4317]
x2: [ 0.299   0.6392  0.0826  0.0823  0.8025  0.7404  0.3861  0.986   0.605
  0.3618  0.761   0.7192  0.5181  0.1091  0.1085  0.2028  0.7049  0.688
  0.8477  0.6176  0.7883  0.6273  0

In [56]:
# create method to train and get result with different settings like online, minibatch, batch and learning rate
def train_and_get_result(x1, x2, lr, batch_size):
    
    if batch_size > 75:
        batch_size = 75
    
    w1 = random.random() * 3
    w2 = random.random() * 3
    b = random.random() * 3
    
    print "random initial w1:\t", w1
    print "random initial w2:\t", w2
    print "random initial b:\t", b
    
    epoch = 0
    # training set size : validation set size = 3 : 1
    training_set_index = 0
    size = len(x1)
    validation_set_index = 0
    validation_set_index_step = size / 4
    pre_validation_error = sys.maxint
    curr_validation_error = 0
    while pre_validation_error >= curr_validation_error or curr_validation_error <= 0: # early stopping when the validation error is greater than its previous one
    #while True:

        validation_set_index += validation_set_index_step
        validation_set_index %= size

        x1_validation = x1[validation_set_index : validation_set_index + validation_set_index_step]
        x2_validation = x2[validation_set_index : validation_set_index + validation_set_index_step]
        y_validation = y[validation_set_index : validation_set_index + validation_set_index_step]

        # training
        i = 1
        for x1i, x2i, yi in zip(x1, x2, y):
            sum_error_w1 = 0
            sum_error_w2 = 0
            sum_error_b = 0
            if x1i not in zip(x1_validation) and x2i not in zip(x2_validation) and yi not in zip(y_validation):
                error = yi - (x1i * w1 + x2i * w2 + b)
                sum_error_w1 += error * x1i
                sum_error_w2 += error * x2i
                sum_error_b += error
                if i >= batch_size: # only update weights after one batch
                    w1 = w1 + lr * sum_error_w1
                    w2 = w2 + lr * sum_error_w2
                    b = b + lr * sum_error_b
                    sum_error_w1 = 0
                    sum_error_w2 = 0
                    sum_error_b = 0
                    i = 1
                    continue
                i += 1

        if epoch >= 4 and epoch % 4 == 0:
            pre_validation_error = curr_validation_error
            if epoch % 100 == 0: # print only every 20 epochs
                print "epoch:\t", epoch, "\t curr_validation_error:\t", curr_validation_error
            curr_validation_error = 0

        # validation
        for x1i, x2i, yi in zip(x1_validation, x2_validation, y_validation):
            curr_validation_error += abs(yi - (x1i * w1 + x2i * w2 + b))

        epoch += 1

    print "Result: pre_validation_error:\t", pre_validation_error, "curr_validation_error:\t", curr_validation_error
    print "Result: w1:\t", w1, "\tw2:\t", w2, "\tb:\t", b, "\tepoch:\t", epoch

In [ ]:
# online
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 1)

In [57]:
# online
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 1)

random initial w1:	1.97215786472
random initial w2:	0.0778104968237
random initial b:	1.49632039726
epoch:	100 	 curr_validation_error:	97.9176316368
epoch:	200 	 curr_validation_error:	76.3871323988
epoch:	300 	 curr_validation_error:	59.8914274128
epoch:	400 	 curr_validation_error:	47.3113163542
epoch:	500 	 curr_validation_error:	37.8756171345
epoch:	600 	 curr_validation_error:	30.728438099
epoch:	700 	 curr_validation_error:	25.8065849644
epoch:	800 	 curr_validation_error:	22.3272506297
epoch:	900 	 curr_validation_error:	19.9748662259
epoch:	1000 	 curr_validation_error:	18.6673248046
epoch:	1100 	 curr_validation_error:	17.8386153236
epoch:	1200 	 curr_validation_error:	17.2376508077
epoch:	1300 	 curr_validation_error:	16.7846731433
epoch:	1400 	 curr_validation_error:	16.5266806468
epoch:	1500 	 curr_validation_error:	16.3879811943
epoch:	1600 	 curr_validation_error:	16.3216754284
epoch:	1700 	 curr_validation_error:	16.2788473834
epoch:	1800 	 curr_validation_error:	16.255

In [30]:
# online
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 1)

random initial w1:	2.3148537341
random initial w2:	1.36285705377
random initial b:	0.794872796773
epoch:	100 	 curr_validation_error:	66.1568845399
epoch:	200 	 curr_validation_error:	41.60403647
epoch:	300 	 curr_validation_error:	27.664522145
epoch:	400 	 curr_validation_error:	20.5093823644
epoch:	500 	 curr_validation_error:	18.0405084603
epoch:	600 	 curr_validation_error:	16.9017782995
epoch:	700 	 curr_validation_error:	16.4536951794
epoch:	800 	 curr_validation_error:	16.3322958825
epoch:	900 	 curr_validation_error:	16.2746297344
epoch:	1000 	 curr_validation_error:	16.2403859051
epoch:	1100 	 curr_validation_error:	16.2241315122
Result: pre_validation_error:	16.2183820217 curr_validation_error:	16.2184630345
Result: w1:	-2.03748976036 	w2:	3.99017762948 	b:	-0.924149464163 	epoch:	1192


In [31]:
# online
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 1)

random initial w1:	2.29474297929
random initial w2:	1.81713755316
random initial b:	0.660486333541
epoch:	100 	 curr_validation_error:	17.8785446695
epoch:	200 	 curr_validation_error:	16.2588554788
Result: pre_validation_error:	16.2441042955 curr_validation_error:	16.2442814427
Result: w1:	-2.03488463737 	w2:	3.99109415209 	b:	-0.924316255564 	epoch:	228


In [36]:
# online
# learning rate lr = 0.01
train_and_get_result(x1, x2, lr = 0.01, batch_size = 1)

random initial w1:	2.69890392308
random initial w2:	2.54365929624
random initial b:	1.39462403815
w1: 2.69607711916
w1: 2.68370835389
w1: 2.64065601851
w1: 2.59371872622
w1: 2.57676779776
w1: 2.5201965899
w1: 2.49206084815
w1: 2.49119640448
w1: 2.46073015788
w1: 2.43530165446
w1: 2.42641318157
w1: 2.42620783064
w1: 2.37729398776
w1: 2.37090059378
w1: 2.36904390908
w1: 2.34877002735
w1: 2.33822276605
w1: 2.32188732923
w1: 2.28567708028
w1: 2.28510321026
w1: 2.28444885929
w1: 2.28182327835
w1: 2.25201913302
w1: 2.24434459861
w1: 2.24289000585
w1: 2.24020309353
w1: 2.23125598972
w1: 2.21745430081
w1: 2.21647480395
w1: 2.17660277229
w1: 2.17549886816
w1: 2.1428889155
w1: 2.11120666318
w1: 2.10933055709
w1: 2.10940499693
w1: 2.10126566191
w1: 2.06579269533
w1: 2.04566096475
w1: 2.01640567497
w1: 2.00313908386
w1: 1.99613996065
w1: 1.97450121231
w1: 1.96732468809
w1: 1.95361877954
w1: 1.94987233609
w1: 1.94990213435
w1: 1.94890235852
w1: 1.93144033613
w1: 1.9185666892
w1: 1.90836123989
w1: 1

w1: 0.173903198878
w1: 0.173348073258
w1: 0.171084278767
w1: 0.16868562912
w1: 0.173661259
w1: 0.171869879243
w1: 0.170479498842
w1: 0.173701904111
w1: 0.175636770521
w1: 0.175758662704
w1: 0.176717617141
w1: 0.174757644207
w1: 0.176586392948
w1: 0.178125232168
w1: 0.177690757035
w1: 0.177998314727
w1: 0.178668269885
w1: 0.1803239905
w1: 0.170870901143
w1: 0.160905239343
w1: 0.162913904591
w1: 0.15454527664
w1: 0.152034169008
w1: 0.153606431805
w1: 0.149948011062
w1: 0.14614690492
w1: 0.148416347469
w1: 0.148624949974
w1: 0.13864125521
w1: 0.13773618609
w1: 0.137839486076
w1: 0.134885280772
w1: 0.136824381664
w1: 0.136613018719
w1: 0.137389995723
w1: 0.138202998782
w1: 0.139805117625
w1: 0.129944383024
w1: 0.129886738023
w1: 0.130370614155
w1: 0.125225270672
w1: 0.127644254826
w1: 0.116379553139
w1: 0.114842662962
w1: 0.108083803829
w1: 0.108016087833
w1: 0.108873635867
w1: 0.112830594504
w1: 0.113643526524
w1: 0.112318082367
w1: 0.112971062667
w1: 0.113727047157
w1: 0.114459433799
w1:

w1: -0.686293685185
w1: -0.690131050392
w1: -0.689200596168
w1: -0.69280495291
w1: -0.693978144917
w1: -0.69301694595
w1: -0.692387258235
w1: -0.702777339527
w1: -0.710330481514
w1: -0.708473437117
w1: -0.706964210086
w1: -0.706239306273
w1: -0.708049227485
w1: -0.707942320305
w1: -0.716709044447
w1: -0.716441474553
w1: -0.719159468493
w1: -0.721681891007
w1: -0.721772576072
w1: -0.721119232961
w1: -0.71995260681
w1: -0.726067769096
w1: -0.730277702422
w1: -0.728611466409
w1: -0.72835191602
w1: -0.728661394191
w1: -0.729895762262
w1: -0.729131497061
w1: -0.728693241977
w1: -0.728926792983
w1: -0.728629354314
w1: -0.728411393057
w1: -0.73102805564
w1: -0.730043999383
w1: -0.731411488428
w1: -0.735199570564
w1: -0.733932414021
w1: -0.741387728968
w1: -0.742473053173
w1: -0.744994559865
w1: -0.744558870333
w1: -0.743948854272
w1: -0.740942885515
w1: -0.740395167893
w1: -0.740927499652
w1: -0.740544555119
w1: -0.739763595303
w1: -0.739262331883
w1: -0.738891181811
w1: -0.738844715645
w1: -

w1: -1.1320415125
w1: -1.13287003674
w1: -1.13344292892
w1: -1.13310479186
w1: -1.13289205637
w1: -1.13218477936
w1: -1.13806267158
w1: -1.13782080167
w1: -1.1376104749
w1: -1.14097144319
w1: -1.14049667511
w1: -1.14635434082
w1: -1.14754923858
w1: -1.14846495334
w1: -1.14802137154
w1: -1.14756525209
w1: -1.1452256503
w1: -1.14492205043
w1: -1.14541102108
w1: -1.14522487777
w1: -1.14447275028
w1: -1.14415781984
w1: -1.14406235508
w1: -1.14447649651
w1: -1.14447254047
w1: -1.14708965569
w1: -1.1465299464
w1: -1.14804674877
w1: -1.14878611412
w1: -1.14799527827
w1: -1.14762932332
w1: -1.15544603663
w1: -1.16094704993
w1: -1.15969044333
w1: -1.15847623967
w1: -1.15769453825
w1: -1.15893875796
w1: -1.15892793377
w1: -1.16520728569
w1: -1.16499286097
w1: -1.16738998882
w1: -1.16923719075
w1: -1.16935863945
w1: -1.16895351292
w1: -1.1679818777
w1: -1.17291840249
w1: -1.17629807627
w1: -1.17391668902
w1: -1.17345897357
w1: -1.1734382672
w1: -1.1748082831
w1: -1.17466927521
w1: -1.17410770874


w1: -1.49273426426
w1: -1.48977056515
w1: -1.4891427966
w1: -1.4888814713
w1: -1.49028653511
w1: -1.49055805316
w1: -1.48987506352
w1: -1.48999361325
w1: -1.48986578857
w1: -1.48966972117
w1: -1.49088612565
w1: -1.48952067074
w1: -1.48989767367
w1: -1.49135155739
w1: -1.49073186193
w1: -1.49071041371
w1: -1.4901147501
w1: -1.48933696095
w1: -1.49063198513
w1: -1.49602205182
w1: -1.49796220197
w1: -1.4979509647
w1: -1.49750286847
w1: -1.49782112264
w1: -1.4993542745
w1: -1.49924059154
w1: -1.49688617362
w1: -1.49738272994
w1: -1.49583157617
w1: -1.49541096796
w1: -1.49536024449
w1: -1.49532591983
w1: -1.49544728902
w1: -1.49455648225
w1: -1.49415049786
w1: -1.49453434389
w1: -1.49372444482
w1: -1.49340721111
w1: -1.49282798024
w1: -1.49389286134
w1: -1.49408550806
w1: -1.49369973191
w1: -1.49629216208
w1: -1.49567166297
w1: -1.49529341561
w1: -1.49720614192
w1: -1.49848350759
w1: -1.49804537288
w1: -1.49803184791
w1: -1.50090666487
w1: -1.50157930091
w1: -1.50157405564
w1: -1.5016869606

...

w1: -1.94462415338
w1: -1.94461271638
w1: -1.94448461334
w1: -1.94457659473
w1: -1.94381531499
w1: -1.94374496118
w1: -1.94406884986
w1: -1.94512544862
w1: -1.94512372821
w1: -1.94559828657
w1: -1.94567211348
w1: -1.94352329006
w1: -1.94349525945
w1: -1.94299930122
w1: -1.94309049914
w1: -1.94639243471
w1: -1.94829693911
w1: -1.94807982482
w1: -1.94796666646
w1: -1.94779203993
w1: -1.94792021476
w1: -1.94731762581
w1: -1.94701042342
w1: -1.94672687333
w1: -1.9491684658
w1: -1.9507716429
w1: -1.95079291446
w1: -1.95045463608
w1: -1.95071068713
w1: -1.95207814205
w1: -1.95129731364
w1: -1.94968175716
w1: -1.94790955177
w1: -1.94817911429
w1: -1.94713857989
w1: -1.94717343373
w1: -1.94714268927
w1: -1.9473692434
w1: -1.94705001102
w1: -1.94644452939
w1: -1.94636185126
w1: -1.94673449566
w1: -1.94581284784
w1: -1.94559674179
w1: -1.94533922684
w1: -1.94415486869
w1: -1.94172492867
w1: -1.941824517
w1: -1.94293807427
w1: -1.94150345569
w1: -1.94145812105
w1: -1.94294518272
w1: -1.9435666969

w1: -1.98082663654
w1: -1.98256730934
w1: -1.98244931966
w1: -1.98426843815
w1: -1.98488954107
w1: -1.98506638464
w1: -1.98511417464
w1: -1.98449859395
w1: -1.98728953328
w1: -1.98915359446
w1: -1.98547559667
w1: -1.98465690137
w1: -1.98403188389
w1: -1.98565508553
w1: -1.98665835778
w1: -1.9858732841
w1: -1.98592331175
w1: -1.98590975065
w1: -1.98573045428
w1: -1.98606525052
w1: -1.98450519605
w1: -1.98424522454
w1: -1.98635703465
w1: -1.98711203688
w1: -1.9893061553
w1: -1.99018454194
w1: -1.98709207423
w1: -1.98624456116
w1: -1.98604634204
w1: -1.98472689693
w1: -1.98473093841
w1: -1.98457097357
w1: -1.98467763589
w1: -1.98391714916
w1: -1.98385991715
w1: -1.98420521283
w1: -1.98529361576
w1: -1.98529203698
w1: -1.98566300562
w1: -1.98576687728
w1: -1.9834362739
w1: -1.98337174011
w1: -1.98289103671
w1: -1.98300532946
w1: -1.98608287767
w1: -1.98781193537
w1: -1.98764552062
w1: -1.9869683321
w1: -1.98607738509
w1: -1.98628184654
w1: -1.98644639127
w1: -1.98815900323
w1: -1.988041511

w1: -1.99602605033
w1: -1.99719106258
w1: -1.99645261143
w1: -1.99640485311
w1: -1.99659092275
w1: -1.99647027972
w1: -1.999587851
w1: -1.99911414227
w1: -1.99908028819
w1: -2.00117850471
w1: -2.00197553866
w1: -2.00410546956
w1: -2.00500116232
w1: -2.00186054933
w1: -2.00101916367
w1: -2.00082739234
w1: -1.99954315382
w1: -1.99955893984
w1: -1.99941194088
w1: -1.99952691859
w1: -1.99876641835
w1: -1.99871716227
w1: -1.99907477312
w1: -2.0001873392
w1: -2.00018579065
w1: -2.00052075482
w1: -2.00065669314
w1: -1.99827414369
w1: -1.99820205042
w1: -1.99772917385
w1: -1.99785596457
w1: -2.00086602749
w1: -2.00253011038
w1: -2.00239738781
w1: -2.00173444349
w1: -2.00084576268
w1: -2.00103035074
w1: -2.00119881676
w1: -2.00282666545
w1: -2.00271104159
w1: -2.00451622742
w1: -2.00510719957
w1: -2.00528537706
w1: -2.00534442574
w1: -2.00473771096
w1: -2.00747560969
w1: -2.00930195744
w1: -2.00559221311
w1: -2.00476457999
w1: -2.00412443377
w1: -2.00575425989
w1: -2.00678600785
w1: -2.00599546

w1: -2.02017395039
w1: -2.019226384
w1: -2.01902502382
w1: -2.0188082841
w1: -2.01725631833
w1: -2.01439801569
w1: -2.01455797261
w1: -2.01539886913
w1: -2.01382156848
w1: -2.01382615856
w1: -2.01522688522
w1: -2.0157337325
w1: -2.01589411736
w1: -2.01594305244
w1: -2.01662380802
w1: -2.01722801196
w1: -2.01725550228
w1: -2.01648889326
w1: -2.01637488145
w1: -2.01753181405
w1: -2.01672967774
w1: -2.01668692727
w1: -2.01687983486
w1: -2.01676795697
w1: -2.01981853651
w1: -2.01933675584
w1: -2.01930584597
w1: -2.01865229076
w1: -2.01775988634
w1: -2.01792350594
w1: -2.01809527189
w1: -2.01963064186
w1: -2.019516921
w1: -2.0212981386
w1: -2.02185437953
w1: -2.02203375149
w1: -2.02210114193
w1: -2.02149864623
w1: -2.02418527139
w1: -2.02597991821
w1: -2.0222318298
w1: -2.02139369252
w1: -2.0207420715
w1: -2.02236721098
w1: -2.02341733482
w1: -2.02261800728
w1: -2.02266297402
w1: -2.02265744158
w1: -2.02247943196
w1: -2.0227466291
w1: -2.02116588648
w1: -2.02085933807
w1: -2.02050046177
w1:

KeyboardInterrupt: 

In [58]:
# minibatch batch_size = 5
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 5)

random initial w1:	2.87555063546
random initial w2:	1.34417371888
random initial b:	2.00174261582
epoch:	100 	 curr_validation_error:	318.755048689
epoch:	200 	 curr_validation_error:	265.315655117
epoch:	300 	 curr_validation_error:	223.150641061
epoch:	400 	 curr_validation_error:	191.120732372
epoch:	500 	 curr_validation_error:	165.554804557
epoch:	600 	 curr_validation_error:	147.750737696
epoch:	700 	 curr_validation_error:	135.200114761
epoch:	800 	 curr_validation_error:	125.410795456
epoch:	900 	 curr_validation_error:	118.864891709
epoch:	1000 	 curr_validation_error:	114.447853644
epoch:	1100 	 curr_validation_error:	111.054345242
epoch:	1200 	 curr_validation_error:	108.410394031
epoch:	1300 	 curr_validation_error:	106.192026954
epoch:	1400 	 curr_validation_error:	104.200313485
epoch:	1500 	 curr_validation_error:	102.469802678
epoch:	1600 	 curr_validation_error:	100.968687807
epoch:	1700 	 curr_validation_error:	99.5734226834
epoch:	1800 	 curr_validation_error:	98.3581

KeyboardInterrupt: 

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 50)

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 5)

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 50)

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 5)

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 50)

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 5)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 50)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 50)

In [ ]:
# batch
